# initialize

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                   // Or use any other 2.x version here

import org.apache.log4j.{Level, Logger}

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)

# Create spark session

In [2]:
import org.apache.spark.sql._
val sparkSession = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
sparkSession.getClass
import sparkSession.sql

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._

sparkSession: SparkSession = org.apache.spark.sql.SparkSession@7c9751
res1_2: Class[T] = class org.apache.spark.sql.SparkSession
import sparkSession.sql

# Load csv

In [3]:
val result = sparkSession.read.format("csv")
      //.option("delimiter", "|")
      .option("header", "true")
      .option("quote", "'")
      .option("nullValue", "NULL")
      .option("inferSchema", "true")
      .load("/home/pi/sparktest/data/transactionlog")

load at cmd2.sc:7

1 / 1

load at cmd2.sc:7

4 / 4

result: DataFrame = [transactionName: string, requestName: string ... 28 more fields]

In [4]:
result.registerTempTable("transaction")
result.printSchema

root
 |-- transactionName: string (nullable = true)
 |-- requestName: string (nullable = true)
 |-- requesterName: string (nullable = true)
 |-- clientTransactionName: string (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- transactionId: long (nullable = true)
 |-- requestID: long (nullable = true)
 |-- responseSize: integer (nullable = true)
 |-- transactionStatus: integer (nullable = true)
 |-- startDateTime: timestamp (nullable = true)
 |-- endDateTime: timestamp (nullable = true)
 |-- executionTime: integer (nullable = true)
 |-- sam_NumberOfInsert: integer (nullable = true)
 |-- sam_TotalTimeOfInsert: long (nullable = true)
 |-- sam_MostExpensiveInsertTime: integer (nullable = true)
 |-- sam_NumberOfUpdate: integer (nullable = true)
 |-- sam_TotalTimeOfUpdate: long (nullable = true)
 |-- sam_MostExpensiveUpdateTime: long (nullable = true)
 |-- sam_NumberOfDelete: integer (nullable = true)
 |-- sam_TotalTimeOfDelete: integer (nullable = true)
 |-- sam_MostExpensiveD

# querying CSV

## Get duration by calculating datetime column

## create transaction duration temp table

In [9]:
val groupByRdd = sql("""
select startDateTime, endDatetime, unix_timestamp(endDateTime,"yyyy-MM-dd HH:mm:ss.SSSS") - unix_timestamp(startDateTime,"yyyy-MM-dd HH:mm:ss.SSSS") as duration from transaction
order by duration desc
""")

groupByRdd.registerTempTable("duration")
// result.select(concat_ws("and","transactionName","requestName")).show(2)

groupByRdd: DataFrame = [startDateTime: timestamp, endDatetime: timestamp ... 1 more field]

## Querying duration temp table

### querying transaction which take least or most time

In [10]:
sql("""
select * from duration order by duration
""").show(false)

show at cmd9.sc:3

3 / 3

+-----------------------+-----------------------+--------+
|startDateTime          |endDatetime            |duration|
+-----------------------+-----------------------+--------+
|2020-07-23 03:31:47.899|2020-07-23 03:31:47.901|0       |
|2020-07-23 03:32:49.698|2020-07-23 03:32:49.699|0       |
|2020-07-23 03:31:47.986|2020-07-23 03:31:47.987|0       |
|2020-07-23 03:30:46.12 |2020-07-23 03:30:46.122|0       |
|2020-07-23 03:31:48.144|2020-07-23 03:31:48.145|0       |
|2020-07-23 03:30:46.275|2020-07-23 03:30:46.276|0       |
|2020-07-23 03:31:48.226|2020-07-23 03:31:48.228|0       |
|2020-07-23 03:30:46.362|2020-07-23 03:30:46.364|0       |
|2020-07-23 03:31:48.382|2020-07-23 03:31:48.383|0       |
|2020-07-23 03:30:46.515|2020-07-23 03:30:46.518|0       |
|2020-07-23 03:31:48.466|2020-07-23 03:31:48.468|0       |
|2020-07-23 03:30:46.655|2020-07-23 03:30:46.657|0       |
|2020-07-23 03:32:12.114|2020-07-23 03:32:12.433|0       |
|2020-07-23 03:30:46.077|2020-07-23 03:30:46.079|0      

### Get aggregate values of duration table

In [27]:
sql("""
select min(duration), max(duration), avg(duration) from duration
""").show

show at cmd26.sc:1

3 / 3

show at cmd26.sc:1

3 / 3

show at cmd26.sc:1

19 / 19

show at cmd26.sc:1

1 / 1

+-------------+-------------+-----------------+
|min(duration)|max(duration)|    avg(duration)|
+-------------+-------------+-----------------+
|            0|         1671|4.358568458568459|
+-------------+-------------+-----------------+



## querying everyday's transaction count

In [25]:
val groupByRdd = sql("""
select to_date(startDateTime,"yyyy-MM-dd") as day, count(*) as sumTxnDay from transaction
group by day
order by sumTxnDay desc
""").show

show at cmd24.sc:1

3 / 3

show at cmd24.sc:1

200 / 200

+----------+---------+
|       day|sumTxnDay|
+----------+---------+
|2020-07-23|    39830|
|2020-07-19|    27503|
|2020-07-22|        7|
+----------+---------+



In [ ]:
val groupByRdd = sql("""
select to_date(startDateTime,"yyyy-MM-dd") as day, count(*) as sumTxnDay from transaction
group by day
order by sumTxnDay desc
""").show

In [29]:
sql("""
select max(responseSize), min(responseSize) from transaction
""").show

show at cmd28.sc:1

3 / 3

show at cmd28.sc:1

1 / 1

+-----------------+-----------------+
|max(responseSize)|min(responseSize)|
+-----------------+-----------------+
|           443906|             2258|
+-----------------+-----------------+

